In [1]:
import pandas as pd
import os
import tensorflow as tf
import tensorflow

from official.nlp import optimization  # to create AdamW optmizer
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_addons as tfa

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/Capgemini/Hackathon/data/best')

Mounted at /content/drive


In [14]:
!pip install tensorflow-text
!pip install -q tf-models-official

     |████████████████████████████████| 1.1MB 17.0MB/s 
     |████████████████████████████████| 1.2MB 54.9MB/s 
     |████████████████████████████████| 645kB 43.4MB/s 
     |████████████████████████████████| 706kB 55.7MB/s 
     |████████████████████████████████| 174kB 57.1MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 37.6MB 79kB/s 
     |████████████████████████████████| 358kB 47.9MB/s 
     |████████████████████████████████| 102kB 14.1MB/s 


In [15]:
import tensorflow_text
# X_test = pd.read_csv('X_test_voting_classifier_new_desc.csv', index_col=['id'])
# fake_X_test = pd.read_csv(os.path.join('..', 'original', 'test.csv'), index_col=['id'])
# Xy_train = pd.read_csv(os.path.join('..', 'original', 'train.csv'), index_col=['id'])
# Xy_train.dropna(subset=['title'], inplace=True)
cleaned_df = pd.read_csv(os.path.join('..', 'all', '400k.csv'), index_col=[0], sep='#')
drop_dup = cleaned_df[cleaned_df['len_title'] >= 2]
drop_dup.drop_duplicates(subset=['title'], inplace=True)
drop_dup.drop_duplicates(subset=['description'], inplace=True)
drop_dup.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,class,title,description,len_description,len_title
0,3,Samsung 32-inch Curved LED Monitor (Ultra- Sli...,1800R curved monitor with 3000:1 contrast rati...,62.0,9.0
1,2,HotHands Body & Hand Super Warmers - Long Last...,"Bring the Heat!,HotHands Body & Hand Super War...",525.0,26.0
4,0,Street Fighter 30th Anniversary Collection - P...,Celebrate the 30th Anniversary of the iconic S...,25.0,10.0
5,1,Pioneer Pet Replacement Pump and Transformer,Replacement pump for Pioneer Pet Fountains. Fo...,22.0,6.0
7,1,ASPCA ACC Cat House & Scratcher w/ Bonus Catni...,Give kitty a relaxing place to play and hide w...,46.0,10.0


In [16]:
epochs = 7
steps_per_epoch = len(drop_dup)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1 * num_train_steps)

optimizer = optimization.create_optimizer(init_lr=3e-5,
                                        num_train_steps=num_train_steps,
                                        num_warmup_steps=num_warmup_steps,
                                        optimizer_type='adamw')

In [20]:
title_model = tf.keras.models.load_model(os.path.join('title_big_bert'), compile=True, custom_objects={'AdamWeightDecay': optimizer})

In [21]:
description_model = tf.keras.models.load_model(os.path.join('..', '..', 'saved_models', 'voting_classifier', 'description_big_data'), compile=True, custom_objects={'AdamWeightDecay': optimizer})

In [22]:
concat_model = tf.keras.models.load_model(os.path.join('concat_big_bert'), compile=True, custom_objects={'AdamWeightDecay': optimizer})

In [ ]:
# fake_X_test = pd.read_csv(os.path.join('..', 'original', 'test.csv'), index_col=['id'])

# # Xy_train['description'].fillna("", inplace=True)
# # Xy_train['concatenation'] = Xy_train['title'] + Xy_train['description']
# fake_X_test['description'].fillna("", inplace=True)
# fake_X_test['concatenation'] = fake_X_test['title'] + fake_X_test['description']

# Xy_train['description'].fillna(Xy_train['title'], inplace=True)
# Xy_train['len_description'] = Xy_train['description'].apply(lambda x: len(x.split(' ')))

# fake_X_test['description'].fillna(fake_X_test['title'], inplace=True)
# fake_X_test['len_description'] = fake_X_test['description'].apply(lambda x: len(x.split(' ')))

# def fill_next(description, len):

#     if len <= 128:
#         return description
#     elif len > 128 and len <= 256:
#         desc_list = description.split(' ')
#         # print(desc_list)
#         desc_list = desc_list[-128:]
#         # print(desc_list)
#         return ' '.join(desc_list)
#     else:
#         desc_list = description.split(' ')[128:256]
#         return ' '.join(desc_list)

# Xy_train['next'] = Xy_train.apply(lambda x: fill_next(x['description'], x['len_description']), axis=1)
# fake_X_test['next'] = fake_X_test.apply(lambda x: fill_next(x['description'], x['len_description']), axis=1)

In [24]:
drop_dup['description'].fillna(drop_dup['title'], inplace=True)
drop_dup['concatenation'] = drop_dup['title'] + drop_dup['description']
drop_dup.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,class,title,description,len_description,len_title,concatenation
0,3,Samsung 32-inch Curved LED Monitor (Ultra- Sli...,1800R curved monitor with 3000:1 contrast rati...,62.0,9.0,Samsung 32-inch Curved LED Monitor (Ultra- Sli...
1,2,HotHands Body & Hand Super Warmers - Long Last...,"Bring the Heat!,HotHands Body & Hand Super War...",525.0,26.0,HotHands Body & Hand Super Warmers - Long Last...
4,0,Street Fighter 30th Anniversary Collection - P...,Celebrate the 30th Anniversary of the iconic S...,25.0,10.0,Street Fighter 30th Anniversary Collection - P...
5,1,Pioneer Pet Replacement Pump and Transformer,Replacement pump for Pioneer Pet Fountains. Fo...,22.0,6.0,Pioneer Pet Replacement Pump and TransformerRe...
7,1,ASPCA ACC Cat House & Scratcher w/ Bonus Catni...,Give kitty a relaxing place to play and hide w...,46.0,10.0,ASPCA ACC Cat House & Scratcher w/ Bonus Catni...


In [43]:
drop_dup.groupby(by='class').count()
subset = drop_dup.groupby(by=['class']).sample(n=30000)
subset_random = subset.sample(frac=1, random_state=42)
print(subset_random.shape)
print(subset_random.head())
subset_random.groupby(by=['class']).count()

(120000, 6)
        class  ...                                      concatenation
329128      2  ...  SRAM PG970 9-Speed CassetteThe SRAM PG-970 9-s...
413247      2  ...  Elite Deluxe 2-3-4 Roller Burgundy/BlackThis d...
229375      1  ...  Absorption Corp Carefresh Pet Bedding, Turquoi...
37948       0  ...  World Championship Poker: All In - Xbox 360In ...
39337       0  ...  Timeshift - PCDr. Aiden Krone is a former prof...

[5 rows x 6 columns]


,title,description,len_description,len_title,concatenation
class,,,,,
0,30000,30000,30000,30000,30000
1,30000,30000,30000,30000,30000
2,30000,30000,30000,30000,30000
3,30000,30000,30000,30000,30000


In [44]:
def get_voting_classifier_df(models, X, names):

    y_pred_all = []
    for idx, model in enumerate(models):
        column = names[idx]
        y_pred_cat = model.predict(X[column], batch_size=32, verbose=1, steps=None, 
                                   callbacks=None, max_queue_size=10,
                                   workers=1, use_multiprocessing=False)
        labels = [column + '_' + str(i) for i in range(len(y_pred_cat[0]))]
        print(labels)
        y_pred_cat_df = pd.DataFrame(y_pred_cat, index=X.index.values, columns=labels)
        y_pred_cat_df.index.rename('id', inplace=True)
        y_pred_all.append(y_pred_cat_df)
    return pd.concat(y_pred_all, axis=1)

models = [title_model, description_model, concat_model]
names = ['title', 'description', 'concatenation']
voting_df = get_voting_classifier_df(models, subset_random, names)
voting_df.to_csv(os.path.join('X_train_voting_classifier_new_desc.csv'))
X_test = pd.read_csv('X_test_voting_classifier_new_desc.csv', index_col=['id'])

3750/3750 [==============================] - 1221s 326ms/step
['title_0', 'title_1', 'title_2', 'title_3']
3750/3750 [==============================] - 1232s 328ms/step
['description_0', 'description_1', 'description_2', 'description_3']
3750/3750 [==============================] - 1246s 332ms/step
['concatenation_0', 'concatenation_1', 'concatenation_2', 'concatenation_3']


In [45]:
subset_random.head()

,class,title,description,len_description,len_title,concatenation
329128,2,SRAM PG970 9-Speed Cassette,The SRAM PG-970 9-speed mountain bike cassette...,19.0,4.0,SRAM PG970 9-Speed CassetteThe SRAM PG-970 9-s...
413247,2,Elite Deluxe 2-3-4 Roller Burgundy/Black,This dynamic bag from Elite gives you the opti...,16.0,5.0,Elite Deluxe 2-3-4 Roller Burgundy/BlackThis d...
229375,1,"Absorption Corp Carefresh Pet Bedding, Turquoi...","Carefresh colors is the safe, healthy and fun ...",150.0,7.0,"Absorption Corp Carefresh Pet Bedding, Turquoi..."
37948,0,World Championship Poker: All In - Xbox 360,In World Championship Poker feat. Howard Leder...,76.0,8.0,World Championship Poker: All In - Xbox 360In ...
39337,0,Timeshift - PC,Dr. Aiden Krone is a former professor of Appli...,203.0,3.0,Timeshift - PCDr. Aiden Krone is a former prof...


In [49]:
X_test.head()

,title_0,title_1,title_2,title_3,desc_0,desc_1,desc_2,desc_3,concat_0,concat_1,concat_2,concat_3,next_0,next_1,next_2,next_3
id,,,,,,,,,,,,,,,,
0,0.925032,0.000875,0.001123,0.072970,0.994496,0.000179,0.000256,0.005069,0.952506,0.000875,0.000830,0.045788,0.965877,0.000642,0.001665,0.031817
1,0.000561,0.927120,0.071119,0.001200,0.000153,0.998839,0.000697,0.000311,0.000956,0.542691,0.451656,0.004697,0.000734,0.997581,0.000990,0.000695
2,0.000013,0.000054,0.999874,0.000061,0.000013,0.000054,0.999874,0.000061,0.000218,0.000368,0.998927,0.000487,0.000393,0.000693,0.998046,0.000867
3,0.000018,0.000021,0.999900,0.000060,0.006398,0.169305,0.821331,0.002966,0.000254,0.001694,0.997359,0.000694,0.001595,0.013067,0.982267,0.003071
4,0.955927,0.000683,0.000733,0.042657,0.971681,0.000345,0.000899,0.027075,0.900738,0.001239,0.001627,0.096395,0.953805,0.001230,0.002269,0.042695


In [50]:
voting_df.head()

,title_0,title_1,title_2,title_3,description_0,description_1,description_2,description_3,concatenation_0,concatenation_1,concatenation_2,concatenation_3
id,,,,,,,,,,,,
329128,0.008239,0.000444,0.000596,0.990722,0.000820,0.000912,0.995475,0.002793,0.000678,0.000499,0.993698,0.005124
413247,0.002824,0.001910,0.927603,0.067663,0.000854,0.000428,0.998326,0.000392,0.000623,0.000362,0.997549,0.001466
229375,0.000210,0.999440,0.000141,0.000209,0.000818,0.998660,0.000364,0.000158,0.000363,0.998695,0.000570,0.000372
37948,0.976938,0.000599,0.001251,0.021213,0.997496,0.000127,0.000250,0.002127,0.976050,0.000850,0.000889,0.022212
39337,0.701902,0.000610,0.000444,0.297045,0.992642,0.001506,0.000655,0.005198,0.829758,0.006839,0.033196,0.130207


In [57]:
import tensorflow_addons as tfa

def train_voting_classifier(X_train, y_train, epochs):

    def build_voting_classifier_model():
        inputs = tf.keras.Input(shape=(12,))
        # x = tf.keras.layers.Dense(, activation=tf.nn.relu)(inputs)
        # x = tf.keras.layers.Dropout(0.2)(x)
        outputs = tf.keras.layers.Dense(4, activation=tf.nn.softmax)(inputs)
        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        return model
    
    voting_classifier_model = build_voting_classifier_model()
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    metrics = tfa.metrics.F1Score(num_classes=4)
  
    voting_classifier_model.compile(optimizer='adam',
                         loss=loss,
                         metrics=metrics)

    history = voting_classifier_model.fit(X_train, y_train_cat, 32, epochs=epochs, validation_split=0.20)

    return voting_classifier_model

In [58]:
voting_df = pd.read_csv(os.path.join('X_train_voting_classifier_new_desc.csv'), index_col='id')
y_train_cat = tf.keras.utils.to_categorical(subset_random['class'])
voting_classifier = train_voting_classifier(voting_df, y_train_cat, epochs=15)

Epoch 1/15
3000/3000 [==============================] - 7s 2ms/step - loss: 0.9778 - f1_score: 0.8696 - val_loss: 0.8068 - val_f1_score: 0.9456
Epoch 2/15
3000/3000 [==============================] - 6s 2ms/step - loss: 0.8044 - f1_score: 0.9462 - val_loss: 0.7977 - val_f1_score: 0.9496
Epoch 3/15
3000/3000 [==============================] - 6s 2ms/step - loss: 0.7978 - f1_score: 0.9499 - val_loss: 0.7955 - val_f1_score: 0.9509
Epoch 4/15
3000/3000 [==============================] - 6s 2ms/step - loss: 0.7965 - f1_score: 0.9499 - val_loss: 0.7947 - val_f1_score: 0.9509
Epoch 5/15
3000/3000 [==============================] - 6s 2ms/step - loss: 0.7964 - f1_score: 0.9490 - val_loss: 0.7943 - val_f1_score: 0.9515
Epoch 6/15
3000/3000 [==============================] - 6s 2ms/step - loss: 0.7956 - f1_score: 0.9496 - val_loss: 0.7940 - val_f1_score: 0.9510
Epoch 7/15
3000/3000 [==============================] - 6s 2ms/step - loss: 0.7937 - f1_score: 0.9514 - val_loss: 0.7938 - val_f1_score:

In [62]:
import numpy as np
X_test_12 = X_test.drop(columns=['next_0', 'next_1', 'next_2', 'next_3'])
y_pred_cat = voting_classifier.predict(X_test_12)
y_pred = np.argmax(y_pred_cat, axis=1)
y_pred_submission = pd.DataFrame(y_pred, index=X_test.index.values, columns=['class'])
y_pred_submission.index.rename('id', inplace=True)
y_pred_submission.to_csv(os.path.join('voting_classifier_new_desc.csv'))